In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [77]:
import collections
import base64
from bs4 import BeautifulSoup
import ia_wayback_dataset
import itertools
import os
from simhash import Simhash
import string

In [165]:
def extract_pure_text_from_html(html):
    soup = BeautifulSoup(html, features='lxml')
    for script in soup(["script", "style"]):
        script.extract()
    return soup.get_text(' ').lower()
    # this frees up memory https://stackoverflow.com/questions/11284643/python-high-memory-usage-with-beautifulsoup
    soup.decompose()
    return text

simhash_size = 128
TRANSLATOR = str.maketrans(string.punctuation, ' '*len(string.punctuation))
def simhash_by_wayback_default(text):
    text = text.translate(TRANSLATOR)
    back_of_words = collections.Counter(text.split())
    return Simhash(back_of_words, simhash_size)

def hamming_distance(a, b):
    return bin(a^b).count('1')

# Ideas to try
- filter stop words
- simhash with bigrams, simhash with shingles, minhash, edit distance, jaccard similarity and etc

In [186]:
import struct
captures = ia_wayback_dataset.load_data(
    url='https://reactjs.org/blog/all.html', year=2018, path='/var/datasets/ia')

def convert_text_to_simhash(capture_data):
    text = extract_pure_text_from_html(capture_data)
    return simhash_by_wayback_default(text)

captures_with_simhash = list(map(lambda c: (c[0], convert_text_to_simhash(c[2]).value), captures))
for (_,simhash_of_capture_previous), (timestamp, simhash_of_capture) in zip(captures_with_simhash, captures_with_simhash[1:]):
    print(timestamp, ' -  hamming distance', hamming_distance(simhash_of_capture_previous, simhash_of_capture))


directory /var/datasets/ia/wbm/urls/a5d099af8ea22a17edc3bc46b6535aeb already exist
created file /var/datasets/ia/wbm/captures/WUAFJGAQZ4X5TBSPQWIE5TP7L6TZIX6I
20180214132613  -  hamming distance 0
20180322205857  -  hamming distance 1
20180330062934  -  hamming distance 2
20180404202601  -  hamming distance 0
20180406020136  -  hamming distance 0
20180414222149  -  hamming distance 0
20180420033338  -  hamming distance 0
20180427021344  -  hamming distance 0
20180504090033  -  hamming distance 0
20180510211558  -  hamming distance 0
20180518141208  -  hamming distance 0
20180526062520  -  hamming distance 0
20180602081826  -  hamming distance 0
20180621233326  -  hamming distance 1
20180628230017  -  hamming distance 0
20180705212030  -  hamming distance 0
20180726161616  -  hamming distance 0
20180802205833  -  hamming distance 1
20180809213815  -  hamming distance 0
20180817024652  -  hamming distance 0
20180824005543  -  hamming distance 0
20180830231238  -  hamming distance 0
20180

fetch IA wayback captures for specific url and year to dataset

In [ ]:
datasets_path = os.environ.get('DATA_SET', '/var/datasets/ia')
set_dataset_path(datasets_path)

store_captures_to_dataset('https://reactjs.org/blog/all.html', 2019)
store_captures_to_dataset('https://reactjs.org/blog/all.html', 2018)

In [ ]:
%ls -la /var/datasets/ia/wbm/captures/